## Based on the [Deep Learning for Coders 2022](https://youtube.com/playlist?list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU) course and the book available on https://github.com/fastai/fastbook## Live demo at https://rafaelaraujo13-chess-checkers.hf.space/

In [2]:
from fastai.vision.all import *
from duckduckgo_search import ddg_images

In [3]:
def search_images(term, max_images=30):
  return L(ddg_images(term, max_results = max_images)).itemgot('image')

In [ ]:
path = Path('images')
search = 'chess', 'checkers'

for s in search:
  dest = (path/s)
  dest.mkdir(exist_ok=True, parents=True)
  download_images(dest, urls=search_images(s, max_images=30))
  resize_images(dest, max_size=500, dest=dest)

failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

In [5]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=1),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=10)

In [ ]:
learner = vision_learner(dls, resnet34, metrics=error_rate)
learner.fine_tune(3)

In [7]:
learner.export('model.pkl')